# Project - Airline AI Assistant

We'll now bring together what we've learned to make an AI Customer Support assistant for an Airline

Airline ticket by using multiple tools

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

# As an alternative, if you'd like to use Ollama instead of OpenAI
# Check that Ollama is running for you locally (see week1/day2 exercise) then uncomment these next 2 lines
# MODEL = "llama3.2"
# openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')


OpenAI API Key exists and begins sk-proj-


In [3]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

## Tools

Tools are an incredibly powerful feature provided by the frontier LLMs.

With tools, you can write a function, and have the LLM call that function as part of its response.

Sounds almost spooky.. we're giving it the power to run code on our machine?

Well, kinda.

There  will be 2 tools at the beginnig. 
1: Will tell the ticket price
2. Will tell the places to see

In [4]:
# 1: Ticket price function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}") # this way we can see this function called in output
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown") # ticket_prices'te bulamazsa Unknown yazıyor
    # return ticket_prices.get(city) # bu boş dönüyor bulamazsa

In [5]:
# get_ticket_price("Berlin")

In [6]:
# There's a particular dictionary structure that's required to describe our function:
# 1st function for first tool

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price,\
                    for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [16]:
# 2: Places to see function

places_to_see = {"london": "London Eye", "paris": "Eyfel", "tokyo": "Tokyo Drift", "berlin": "I don't know", \
                "istanbul": "Şükrü Saraçoğlu, Fenerbahçe Stadium", "bayburt": "Ortugu. \
                This place is where onur has born. Everybody should see there."}

def get_places_to_see(destination_city): 
    print(f"Tool get_places_to_see called for {destination_city}") # this way we can see this function called in output
    city = destination_city.lower()
    return places_to_see.get(city, "Bimiyorum ki valla Onur'a sor")

In [8]:
# get_places_to_see("Bayburt")

In [9]:
# get_places_to_see("Ankara")

In [10]:
# There's a particular dictionary structure that's required to describe our function:
# 2nd function for second tool

place_function = {
    "name": "get_places_to_see",
    "description": "Get the place to see in the destination city. \
                    Call this whenever you need to know the place to visit,\
                    for example when a customer asks 'Where can I visit in this city' or 'Where should I see in this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [11]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function}, 
        {"type": "function", "function": place_function}]

## Getting OpenAI to use our Tool

There's some fiddly stuff to allow OpenAI "to call our tool"

What we actually do is give the LLM the opportunity to inform us that it wants us to run the tool.

Here's how the new chat function looks:

In [12]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    print(f"finish reason {response.choices[0].finish_reason}")
    print(f"Response {response.choices[0].message.content}")

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        print(f" \n, Tool call message: {message}, \n")
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [13]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    tool_call = message.tool_calls[0] # message already has which tool to call
    arguments = json.loads(tool_call.function.arguments) # because it is in json format
    city = arguments.get('destination_city')
    function_name = tool_call.function.name # this is just a string, look at message(call tool message)
    
    # price = get_ticket_price(city) we dont know which function to use
    func = 0
    if function_name == "get_ticket_price":
        price = get_ticket_price(city)
        func = 1
    elif function_name == "get_places_to_see":
        place = get_places_to_see(city)
        func = 2
        
    if func == 1:
        response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}), # destination_city tool call messagının içinde var
        "tool_call_id": tool_call.id
        }
    elif func == 2:
        response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"place": place}), # destination_city tool call messagının içinde var
        "tool_call_id": tool_call.id
        }
        
    return response, city

In [17]:
gr.ChatInterface(fn=chat, type="messages").launch()

# call id'de hata alıyor. şehir ismi verip taskı tam anlatmazsan karıştırıyor.

* Running on local URL:  http://127.0.0.1:7876

To create a public link, set `share=True` in `launch()`.


finish reason tool_calls
Response None
 
, Tool call message: ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_xVbkcYYOWCnJgZB0KxrbdFRl', function=Function(arguments='{"destination_city":"Bayburt"}', name='get_places_to_see'), type='function')]), 

Tool get_places_to_see called for Bayburt
